## Reminders of syntax with the facile library

<div class="alert alert-success">
    <b>Hint</b>: Constraints may be summed: the result is the number of True values in the set of constraints.<br/>(However, this doesn't work with global constraints such as alldifferent)
</div>

Example: Find an array of length $n$ taking values in $[0, n-1]$ so that the $i$-th value is equal to the number of $i$ inside the array.

In [10]:
import facile
n = 10
array = [facile.variable(range(n)) for i in range(n)]
for i in range(n):
    facile.constraint(sum([x == i for x in array]) == array[i])

if facile.solve(array):
    print(f"indices: {list(range(n))}")
    print(f"array:   {[v.value() for v in array]}")

indices: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
array:   [6, 2, 1, 0, 0, 0, 1, 0, 0, 0]


<div class="alert alert-success">
    <b>Hint</b>: You may use the & and | operators for the logical $\land$ and $\lor$.<br/>(However, this doesn't work with global constraints such as alldifferent)
</div>

In [13]:
a, b = facile.variable([0, 1, 2]), facile.variable([0, 1, 2])

left = (a == b) & (a + b == 2)
right = a > b
facile.constraint(left | right)
result = facile.solve([a, b])
a.value(), b.value()

(1, 0)

## Picross

Picross is a popular game of a similar nature to Sudoku.  
It consists of a grid of square dimension $n \times n$ and of a list of integers for each row and each column.

![](./picross_duck.png)

We can use the following notations:

- $b_{i,j} \in \{0, 1\}$ represents a bit, indicating a column or a row where $i, j \in \{1, ..., n\}$ represents a column or row index;
- $m_i \in \{1, ..., \lceil n/2 \rceil\}$ represents the size of the list associated to row/column $i$;
- $a_{i,k} \in \{1, ..., n\}$ is the $k$-th integer of the list associated to row/column $i$.

The goal is to darken some cells of the grid, such that in each row/column $i$ the number of successive black cells form the list $\{a_{i,1}, ..., a_{i,m_i}\}$.

<div class="alert alert-warning">
    <b>Paper submission</b> (half of the grade):<br/>
Before heading to the code, you shall answer the following questions.<br/>
    Please <b>handwrite</b> your answers on a separate sheet of paper. You will submit a pdf scan of this document together with your notebook.  
(You may take a picture with your phone if need be but the pdf option is preferred.)
</div>

1. There are two ways to model the problem:
    - binary variables (1) associated to the color of each cell, or
    - integer variables (2) associated to the bottom/left coordinate of each black cluster.  
  For **both** ways, specify the decision variables and their associated domain.

2. Write the constraint stating that on each line and column, two blocks do not overlap.  
Choose **one** of the two ways of modelling the problem (i.e. decision variables from (1) or from (2)) to express this constraint.

3. Write the constraint that puts a limit to the number of black cells on each line/column.  
Choose **one** of the two ways of modelling the problem (i.e. decision variables from (1) or from (2)) to express this constraint.

4. Write the constraint that links decision variables from (1) to decision variables from (2), stating the conditions under which a cell must be coloured in black.

<div class="alert alert-danger">
    Before you start coding, <b>please take time to understand</b> what happened: we stated two different ways to model the same problem (question 1.). Both are convenient for stating one part of the problem, but cannot state the whole set of constraints (question 2. and 3.).<br/>One constraint (question 4.) helps linking both decision variables.
</div>

You are provided with a set of problems in the Python dictionary `picross`. Your code will be evaluated on all these problems, but you should start with small problems like `moon`, `star` and `cat`. The most time-consuming should be `house` so do not try it before your code is stable.

Have a look at variables `lines` and `columns`, since you will have to feed this data into your problem. From now, it should be (almost) direct to code the constraints you have written on paper into Python.

<div class="alert alert-warning">
    Finally, <b>do not waste too much time on the output</b>.  
    You will <b>not</b> get extra points with a fancy matplotlib plot.  
    A text output, with stars (<code>*</code>) or black characters (<code>■</code>) like the following ones is enough:
</div>

```
         ***                 ■■■    
        *****               ■■■■■   
       **** ***            ■■■■ ■■■ 
       *******             ■■■■■■■  
        *****               ■■■■■   
         ***                 ■■■    
        *****               ■■■■■   
*     ********      ■     ■■■■■■■■  
***  ***   ***      ■■■  ■■■   ■■■  
******* *** **      ■■■■■■■ ■■■ ■■  
 ***** **** **       ■■■■■ ■■■■ ■■  
 ********  **        ■■■■■■■■  ■■   
  **********          ■■■■■■■■■■    
    ** ***              ■■ ■■■      
      ******              ■■■■■■    

```

In [15]:
from picross import picross

# Choose among 'moon', 'star', 'cat', 'horse', 'house' and 'duck'
lines, columns = picross['moon']

print(lines)
print(columns)

n_l, n_c = len(lines), len(columns)

(n_l, n_c)

[[2], [2], [1, 2], [5], [3]]
[[2], [2], [1, 2], [5], [3]]


(5, 5)

In [179]:
import facile
b = [[facile.variable([0,1]) for i in range(n_l)] for j in range(n_c)])
l = [[facile.variable(range(n_l)) for j in range(len(lines[i]))] for i in range(n_l)]
c = [[facile.variable(range(n_c)) for j in range(len(columns[i]))] for i in range(n_c)]

for i in range(n_l):
    facile.constraint(sum(b[i])==sum(lines[i]))
for j in range(n_c):
    facile.constraint(sum([b[i][j] for i in range(n_l)])==sum(columns[j]))
for i in range(n_l):
    for k in range(len(lines[i])-1):
        facile.constraint(l[i][k+1]-l[i][k]>lines[i][k])
for j in range(n_c):
    for k in range(len(columns[j])-1):
        facile.constraint(c[j][k+1]-c[j][k]>columns[j][k]) 
for i in range(n_l):
    for k in range(len(lines[i])):
        facile.constraint(b[i][l[i][k].value()]==1)
for j in range(n_c):
    for k in range(len(columns[j])):
        facile.constraint(b[c[k][j].value()][j]==1)
white = ' '
black = "*"

if facile.solve([b_ij for b_i in b for b_ij in b_i]+[l_ik for l_i in l for l_ik in l_i]+[c_ik for c_i in c for c_ik in c_i]):
    for i in range(n_l):
        str = ''
        for j in range(n_c):
            if b[i][j].value()==0:
                str += white
            else:
                str += black
        print(str)

TypeError: The arguments must be variables or expressions

In [ ]:
m_l = [len(lines[i]) for i in range(n_l)]
m_c = [len(columns[j]) for j in range(n_c)]
a_l = [[lines[i][k] for k in range(m_l[i])] for i in range(n_l)]
a_c = [[columns[j][k] for k in range(m_c[j])] for j in range(n_c)]

In [167]:
from facile import variable, constraint, solve

# Number of buckets
nb = 3
# Number of steps (let's say we know... :p)
steps = 8
# The capacity of each bucket
capacity = [8, 5, 3]

buckets = [[variable(range(capacity[b] + 1)) for b in range(nb)]
           for i in range(steps)]

constraint(buckets[0][0] == 8)
constraint(buckets[0][1] == 0)
constraint(buckets[0][2] == 0)

constraint(buckets[steps - 1][0] == 4)
constraint(buckets[steps - 1][1] == 4)
constraint(buckets[steps - 1][2] == 0)

for i in range(steps - 1):
    # we change the contents of two buckets at a time
    constraint(sum([buckets[i][b] != buckets[i+1][b] for b in range(nb)]) == 2)
    # we play with a constant amount of water
    constraint(sum([buckets[i][b] for b in range(nb)]) == 8)
    for b1 in range(nb):
        for b2 in range(b1):
            constraint(
                # either the content of the bucket does not change
                (buckets[i][b1] == buckets[i+1][b1]) |
                (buckets[i][b2] == buckets[i+1][b2]) |
                # or the bucket ends up empty or full
                (buckets[i+1][b1] == 0) | (buckets[i+1][b1] == capacity[b1]) |
                (buckets[i+1][b2] == 0) | (buckets[i+1][b2] == capacity[b2])
                )

if solve([b for sub in buckets for b in sub]):
    for sub in buckets:
        print ([b.value() for b in sub])

[8, 0, 0]
[3, 5, 0]
[3, 2, 3]
[6, 2, 0]
[6, 0, 2]
[1, 5, 2]
[1, 4, 3]
[4, 4, 0]


In [125]:
import facile
b = []
for i in range(n_l):
    b.append(facile.array([facile.variable([0,1]) for i in range(n_l)]))
for i in range(n_l):
    facile.constraint(sum(b[i])==sum(lines[i]))
for j in range(n_c):
    facile.constraint(sum([b[i][j] for i in range(n_l)])==sum(columns[j]))
facile.solve([[b[i] for i in range(n_l)] for j in range(n_l)])

AssertionError: All arguments must be variables

In [142]:
import facile
b_l = facile.array([facile.variable([0,1]) for i in range(n_l)])
b_c = facile.array([facile.variable([0,1]) for j in range(n_c)])
for i in range(n_l):
    facile.constraint(sum([b_l[i]*b_c[j] for j in range(n_c)]) == sum(lines[i]))
# # for j in range(n_c):
# #     facile.constraint(sum([b_l[i]*b_c[j] for i in range(n_l)]) == sum(columns[j]))
    
facile.solve(list(b_l)+list(b_c))
print(b_l.value)
print(b_c.value)

ValueError: The problem is overconstrained

In [144]:
import facile
b = facile.array([facile.variable([0,1]) for i in range(n_l*n_c)])

facile.solve(b)

True